In [1]:
# ################# I M P O R T S ################# #
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ################# S E T T I N G S ################# #
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


Daha önce üzerinde veri ön işleme ve bazı özellik mühendisliği işlemleri yapılıp csv dosyası şeklinde diske kaydedilen dosyamızı bir Pandas dataframe şeklinde içe aktarıyoruz.

In [2]:
df=pd.read_csv(r"data/step6_remastered.csv")

## ENCODING

Makine öğrenmesinin olmazsa olmazı kategorik türdeki predictor değişkenleri makinelerin anlayabileceği sayısal temsillerine çevirmektir. Bu işleme encoding denilmektedir.

In [3]:
# Sınıflar arası fark varsa (ordinal kategorik değişkenler)- Ordinal Encoding
ord_cols = ["LEVEL", "EXPERIENCE"]

# Sınıflar arası fark yoksa (nominal kategorik değişkenler) - Dummy Encoding
dummy_cols = ["POSITION", "GENDER", "COMPANY", "WORK_TYPE", "CITY", "CURRENCY"]

In [4]:
# Veri setimizin orijinaline dokunmamak için kopyasını oluşturalım
df_enc = df.copy(deep=True)

### 1. Ordinal Encoding

In [5]:
# Encoding işleminin alfabetik değil, küçükten büyüğe olmasını istiyoruz. Bu amaçla her değişken için ayrı sıralı liste oluşturuyoruz
lvl_order = ['Junior', 'Middle', 'Senior']
exp_order = ['0 - 1 Yıl', '1 - 3 Yıl', '3 - 5 Yıl', '5 - 7 Yıl', '7 - 10 Yıl', '10 - 14 Yıl', '15 Yıl ve üzeri']

# Ordinal Encoder nesnesi oluştur, sıralamaları belirt, çıktı veri türü olarak integer seç
ord_enc = OrdinalEncoder(categories=[lvl_order, exp_order], dtype=int)

# Encoding işlemini hazırlayıp değişkenlere uygula
df_enc[ord_cols] = ord_enc.fit_transform(df[ord_cols])


In [6]:
# Bakalım sınıflar istediğimiz sırada ve şekilde oluşmuş mu
ord_enc.categories_

[array(['Junior', 'Middle', 'Senior'], dtype=object),
 array(['0 - 1 Yıl', '1 - 3 Yıl', '3 - 5 Yıl', '5 - 7 Yıl', '7 - 10 Yıl',
        '10 - 14 Yıl', '15 Yıl ve üzeri'], dtype=object)]

In [7]:
# Asıl veri setinden sabit bir örneklem alalım
df.sample(10, random_state=40)

,LEVEL,POSITION,EXPERIENCE,GENDER,COMPANY,WORK_TYPE,CITY,CURRENCY,SALARY_AVG_TL
3430,Middle,FULL STACK DEVELOPER,3 - 5 Yıl,Erkek,Kurumsal,Remote,yurt_ici,TRY,17499
3959,Junior,DATA SCIENTIST,0 - 1 Yıl,Erkek,Startup,Ofis,yurt_ici,TRY,6499
4985,Junior,FRONT-END DEVELOPER,1 - 3 Yıl,Kadın,Startup,Hibrit,yurt_ici,TRY,16499
2333,Middle,BACK-END DEVELOPER,3 - 5 Yıl,Erkek,Startup,Remote,yurt_ici,TRY,17499
2733,Middle,BACK-END DEVELOPER,1 - 3 Yıl,Erkek,Kurumsal,Geçici Remote,yurt_ici,TRY,14499
118,Middle,GAME DEVELOPER,3 - 5 Yıl,Erkek,Oyun,Remote,yurt_ici,TRY,25499
70,Middle,TEAM / TECH LEAD,3 - 5 Yıl,Erkek,Kurumsal,Hibrit,yurt_ici,TRY,22499
147,Junior,QA / TEST AUTOMATION DEVELOPER,1 - 3 Yıl,Erkek,Kurumsal,Hibrit,yurt_ici,TRY,10499
2603,Senior,TEAM / TECH LEAD,15 Yıl ve üzeri,Erkek,Kurumsal,Remote,yurt_ici,TRY,25499
4774,Junior,QA / TEST AUTOMATION DEVELOPER,0 - 1 Yıl,Erkek,Startup,Remote,yurt_ici,TRY,7499


In [8]:
# Aynı örneklemin ordinal encoding uygulanmış şekli
df_enc.sample(10, random_state=40)

,LEVEL,POSITION,EXPERIENCE,GENDER,COMPANY,WORK_TYPE,CITY,CURRENCY,SALARY_AVG_TL
3430,1,FULL STACK DEVELOPER,2,Erkek,Kurumsal,Remote,yurt_ici,TRY,17499
3959,0,DATA SCIENTIST,0,Erkek,Startup,Ofis,yurt_ici,TRY,6499
4985,0,FRONT-END DEVELOPER,1,Kadın,Startup,Hibrit,yurt_ici,TRY,16499
2333,1,BACK-END DEVELOPER,2,Erkek,Startup,Remote,yurt_ici,TRY,17499
2733,1,BACK-END DEVELOPER,1,Erkek,Kurumsal,Geçici Remote,yurt_ici,TRY,14499
118,1,GAME DEVELOPER,2,Erkek,Oyun,Remote,yurt_ici,TRY,25499
70,1,TEAM / TECH LEAD,2,Erkek,Kurumsal,Hibrit,yurt_ici,TRY,22499
147,0,QA / TEST AUTOMATION DEVELOPER,1,Erkek,Kurumsal,Hibrit,yurt_ici,TRY,10499
2603,2,TEAM / TECH LEAD,6,Erkek,Kurumsal,Remote,yurt_ici,TRY,25499
4774,0,QA / TEST AUTOMATION DEVELOPER,0,Erkek,Startup,Remote,yurt_ici,TRY,7499


### 2. Dummy Encoding

In [9]:
# Encoding yapılmamış bütün kategorik değişkenlere Dummy Encoding uygula
df_enc = pd.get_dummies(df_enc, drop_first=True)

In [10]:
df_enc.head()

,LEVEL,EXPERIENCE,SALARY_AVG_TL,POSITION_CTO,POSITION_DATA SCIENTIST,POSITION_DATABASE ADMIN,POSITION_DEVOPS ENGINEER,POSITION_EMBEDDED SOFTWARE DEVELOPER,POSITION_FRONT-END DEVELOPER,POSITION_FULL STACK DEVELOPER,...,COMPANY_Startup,COMPANY_Yazılım & Teknoloji,WORK_TYPE_Geçici Remote,WORK_TYPE_Hibrit,WORK_TYPE_Ofis,WORK_TYPE_Remote,CITY_yurt_ici,CURRENCY_GBP,CURRENCY_TRY,CURRENCY_USD
0,1,2,15499,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,1,0
1,0,0,8499,0,0,0,0,0,1,0,...,1,0,0,1,0,0,1,0,1,0
2,1,1,15499,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
3,0,1,10499,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,0,1,0
4,0,0,5499,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


## Değişken Sırasını Ayarlama

In [11]:
# Değişkenler için bir liste oluşturalım
cols = df_enc.columns.tolist()
cols

['LEVEL',
 'EXPERIENCE',
 'SALARY_AVG_TL',
 'POSITION_CTO',
 'POSITION_DATA SCIENTIST',
 'POSITION_DATABASE ADMIN',
 'POSITION_DEVOPS ENGINEER',
 'POSITION_EMBEDDED SOFTWARE DEVELOPER',
 'POSITION_FRONT-END DEVELOPER',
 'POSITION_FULL STACK DEVELOPER',
 'POSITION_GAME DEVELOPER',
 'POSITION_MOBILE APPLICATION DEVELOPER',
 'POSITION_OTHER',
 'POSITION_QA / TEST AUTOMATION DEVELOPER',
 'POSITION_SOFTWARE ARCHITECT',
 'POSITION_SOFTWARE DEVELOPMENT MANAGER',
 'POSITION_TEAM / TECH LEAD',
 'GENDER_Kadın',
 'COMPANY_Dijital / Reklam Ajansı',
 'COMPANY_E-Ticaret',
 'COMPANY_Fintech / Finans',
 'COMPANY_Kurumsal',
 'COMPANY_Outsource',
 'COMPANY_Oyun',
 'COMPANY_Startup',
 'COMPANY_Yazılım & Teknoloji',
 'WORK_TYPE_Geçici Remote',
 'WORK_TYPE_Hibrit',
 'WORK_TYPE_Ofis',
 'WORK_TYPE_Remote',
 'CITY_yurt_ici',
 'CURRENCY_GBP',
 'CURRENCY_TRY',
 'CURRENCY_USD']

In [12]:
# Üçüncü sıradaki (2. indeks) elemanı son konuma taşı
cols = cols[:2] + cols[3:] + cols[2:3]
cols

['LEVEL',
 'EXPERIENCE',
 'POSITION_CTO',
 'POSITION_DATA SCIENTIST',
 'POSITION_DATABASE ADMIN',
 'POSITION_DEVOPS ENGINEER',
 'POSITION_EMBEDDED SOFTWARE DEVELOPER',
 'POSITION_FRONT-END DEVELOPER',
 'POSITION_FULL STACK DEVELOPER',
 'POSITION_GAME DEVELOPER',
 'POSITION_MOBILE APPLICATION DEVELOPER',
 'POSITION_OTHER',
 'POSITION_QA / TEST AUTOMATION DEVELOPER',
 'POSITION_SOFTWARE ARCHITECT',
 'POSITION_SOFTWARE DEVELOPMENT MANAGER',
 'POSITION_TEAM / TECH LEAD',
 'GENDER_Kadın',
 'COMPANY_Dijital / Reklam Ajansı',
 'COMPANY_E-Ticaret',
 'COMPANY_Fintech / Finans',
 'COMPANY_Kurumsal',
 'COMPANY_Outsource',
 'COMPANY_Oyun',
 'COMPANY_Startup',
 'COMPANY_Yazılım & Teknoloji',
 'WORK_TYPE_Geçici Remote',
 'WORK_TYPE_Hibrit',
 'WORK_TYPE_Ofis',
 'WORK_TYPE_Remote',
 'CITY_yurt_ici',
 'CURRENCY_GBP',
 'CURRENCY_TRY',
 'CURRENCY_USD',
 'SALARY_AVG_TL']

In [13]:
# Yeni sıralamayı veri setine uygula
df_enc = df_enc[cols]

In [14]:
df_enc.head()   # Bingo !

,LEVEL,EXPERIENCE,POSITION_CTO,POSITION_DATA SCIENTIST,POSITION_DATABASE ADMIN,POSITION_DEVOPS ENGINEER,POSITION_EMBEDDED SOFTWARE DEVELOPER,POSITION_FRONT-END DEVELOPER,POSITION_FULL STACK DEVELOPER,POSITION_GAME DEVELOPER,...,COMPANY_Yazılım & Teknoloji,WORK_TYPE_Geçici Remote,WORK_TYPE_Hibrit,WORK_TYPE_Ofis,WORK_TYPE_Remote,CITY_yurt_ici,CURRENCY_GBP,CURRENCY_TRY,CURRENCY_USD,SALARY_AVG_TL
0,1,2,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,15499
1,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,8499
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,15499
3,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,1,0,10499
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,5499


## Variance Inflation Factor (VIF)

Multicollinearity (çoklu eşdoğrusallık) durumunu kontrol etmenin yaygın yöntemlerinden biri varyans büyütme faktörüdür. (VIF - Variance Inflation Factor):

    VIF=1, Çok az
    VIF<5, Orta derecede
    VIF>5, Aşırı derecede (Kaçınmamız gereken senaryo)

Şimdi, VIF skorlarını hesaplayalım:

In [15]:
non_dummy = ['SALARY_AVG_TL', 'EXPERIENCE']
cols2 = [col for col in cols if col not in non_dummy]

# Dummy encoding uygulanmış bağımsız değişken seti
X = df_enc[cols2]

In [16]:
# VIF hesaplaması yapan fonksiyonumuz
def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Feature', 'VIF'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'VIF', axis = 0, ascending=False, inplace=False)
    
X=df_enc.drop(non_dummy, axis=1)
calculate_vif(X)

,Feature,VIF
0,LEVEL,3.62000
21,COMPANY_Oyun,2.32000
8,POSITION_GAME DEVELOPER,2.03000
16,COMPANY_Dijital / Reklam Ajansı,1.60000
29,CURRENCY_GBP,1.18000
13,POSITION_SOFTWARE DEVELOPMENT MANAGER,1.15000
23,COMPANY_Yazılım & Teknoloji,1.14000
5,POSITION_EMBEDDED SOFTWARE DEVELOPER,1.14000
12,POSITION_SOFTWARE ARCHITECT,1.13000
2,POSITION_DATA SCIENTIST,1.12000


**NOT:** Başta One Hot Encoding metodunu denemiştik. O yöntemde ilk sınıf için de encoding uygulandığından VIF skorları çok yüksek çıkmıştı.
Daha sonra Dummy Encoding uygulayıp drop_first=True argümanı ile her değişkenin ilk sınıfını düşürdüğümüz için Variance Inflation Factor (VIF) değerleri 5'ten küçük çıktı. Yani, artık Lineer Regression da kullanabiliriz!

In [17]:
# Veri setini dosya şeklinde dışarı aktarmadan önce son bir kez kontrol edelim
df_enc.tail(15)

,LEVEL,EXPERIENCE,POSITION_CTO,POSITION_DATA SCIENTIST,POSITION_DATABASE ADMIN,POSITION_DEVOPS ENGINEER,POSITION_EMBEDDED SOFTWARE DEVELOPER,POSITION_FRONT-END DEVELOPER,POSITION_FULL STACK DEVELOPER,POSITION_GAME DEVELOPER,...,COMPANY_Yazılım & Teknoloji,WORK_TYPE_Geçici Remote,WORK_TYPE_Hibrit,WORK_TYPE_Ofis,WORK_TYPE_Remote,CITY_yurt_ici,CURRENCY_GBP,CURRENCY_TRY,CURRENCY_USD,SALARY_AVG_TL
5011,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,6499
5012,2,2,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,17499
5013,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,7499
5014,2,2,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,1,0,15499
5015,2,4,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,32499
5016,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,1,0,9499
5017,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,83825
5018,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,11499
5019,2,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,25499
5020,2,2,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,46565


In [ ]:
# Encode edilmiş veri setini csv dosyası şeklinde kaydedelim.
df_enc.to_csv("data/step6_encoded_remastered.csv", encoding='utf-8', index=False)